# ACID upload of data objects

Here we explore the ACID behavior of LaminDB's upload API.

In [ ]:
from lndb_setup import login, init

# from lndb_setup._clone import setup_local_test_postgres

login(user="testuser1")
init(storage="acidtests", schema="bionty")
# pgurl = setup_local_test_postgres()
# init(storage="acidtests", schema="bionty", db=pgurl)

In [ ]:
import lamindb as ln
import pytest

ln.nb.header()

Let's try to ingest a heavily-linked data object with a corrupted storage location.

In [ ]:
# TODO: test for an unsuccesful data object upload.

# import lamindb.knowledge as lnk
# from lndb_setup import settings, load
# from cloudpathlib import CloudPath

# # Create data object linked to 10,044 gene records
# adata = ln.dev.datasets.anndata_mouse_sc_lymph_node()
# reference = ln.knowledge.Gene(
#     id=lnk.lookup.gene_id.ensembl_gene_id,
#     species=lnk.lookup.species.mouse,
# )
# dobject = ln.DObject(adata, name="Mouse Lymph Node scRNA-seq", features_ref=reference)

# # Update storage path with corrupt path
# settings.instance.storage._root = CloudPath("s3://nf-core-awsmegatests")

# # Ingest data object
# added_dobject = ln.add(dobject)

In [ ]:
# # Restore storage path
# load("acidtests")

If a list of data objects is passed to `ln.add()` and the upload of one of these data objects fails, the successful uploads are maintained and a `RuntimeError` is raised, listing the successfully uploaded data objects.

In [ ]:
# TODO: test for an unsuccesful DB transaction. The current example only shows a normal traceback being returned for an error unrelated to the DB connection.
from sqlalchemy.orm.exc import UnmappedInstanceError

filepath = ln.dev.datasets.file_jpg_paradisi05()
dobject = ln.DObject(filepath)
dobjects = [dobject, "this is not a data object"]
with pytest.raises(UnmappedInstanceError) as e:
    exception = e
    ln.add(dobjects)
print(exception.exconly())